# 🎯 Unsupervised Attack Clustering: Jamming vs Spoofing

**Мета**: Автоматична класифікація типів атак на GNSS (Jamming / Spoofing) без використання мітки типу атаки.

## Фізика явищ:
- **Jamming**: Низький CNO (сигнал заглушений), але супутники "видно" в небі
- **Spoofing**: Нормальний або високий CNO (підроблений сигнал), але помилки позиціонування

## Методологія:
1. K-Means кластеризація на аномальних точках
2. Автоматичне визначення кластера Jamming (мін. CNO)
3. Візуалізація розділення

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
print("✅ Бібліотеки імпортовано")

## 1. Завантаження даних та фільтрація аномалій

In [ ]:
DATA_PATH = '../data/processed/all_data_compressed.parquet'
df = pd.read_parquet(DATA_PATH)

# Фільтруємо ТІЛЬКИ аномальні точки (атаки)
df_anomalies = df[df['overallPositionLabel'] == 1].copy()
print(f"📊 Загальна кількість записів: {len(df):,}")
print(f"🎯 Аномальних точок (атаки): {len(df_anomalies):,} ({len(df_anomalies)/len(df)*100:.2f}%)")

## 2. Feature Engineering для кластеризації

In [ ]:
# Sat efficiency - співвідношення використаних до видимих супутників
if 'numSV' in df_anomalies.columns and 'numSatsTracked' in df_anomalies.columns:
    df_anomalies['sat_efficiency'] = df_anomalies['numSV'] / df_anomalies['numSatsTracked']
    df_anomalies['sat_efficiency'] = df_anomalies['sat_efficiency'].replace([np.inf, -np.inf], np.nan).fillna(0)
else:
    df_anomalies['sat_efficiency'] = 0

# Log-transform vAcc для кращого розподілу
if 'vAcc' in df_anomalies.columns:
    df_anomalies['vAcc_log'] = np.log1p(df_anomalies['vAcc'])
else:
    df_anomalies['vAcc_log'] = 0

# Ознаки для кластеризації
clustering_features = ['cnoMean', 'numSatsTracked', 'sat_efficiency', 'vAcc_log']

# Заповнення пропусків
for col in clustering_features:
    if col in df_anomalies.columns:
        df_anomalies[col] = df_anomalies[col].fillna(0)

print(f"✅ Підготовлено {len(clustering_features)} ознак для кластеризації")
df_anomalies[clustering_features].describe().round(2)

## 3. K-Means кластеризація

In [ ]:
# Масштабування
X = df_anomalies[clustering_features].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-Means з k=2 (Jamming vs Spoofing)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df_anomalies['attack_type_cluster'] = kmeans.fit_predict(X_scaled)

print(f"✅ Кластеризація завершена")
print(f"   Inertia (сума квадратів відстаней): {kmeans.inertia_:,.0f}")

## 4. Автоматичне визначення типу атаки

**Логіка**:
- Кластер з **нижчим cnoMean** = Jamming (сигнал заглушений)
- Кластер з **вищим cnoMean** = Spoofing (підроблений сигнал)

In [ ]:
# Статистика кластерів
cluster_stats = df_anomalies.groupby('attack_type_cluster').agg({
    'cnoMean': 'mean',
    'vAcc': 'mean',
    'sat_efficiency': 'mean'
})

print("📊 Статистика кластерів:")
display(cluster_stats.round(2))

# Автоматичне призначення міток
jamming_cluster = cluster_stats['cnoMean'].idxmin()  # Мін. CNO = Jamming
spoofing_cluster = cluster_stats['cnoMean'].idxmax() # Макс. CNO = Spoofing

cluster_map = {
    jamming_cluster: 'Jamming',
    spoofing_cluster: 'Spoofing'
}

df_anomalies['attack_type'] = df_anomalies['attack_type_cluster'].map(cluster_map)

print(f"\n🔴 Jamming cluster: {jamming_cluster} (avg CNO: {cluster_stats.loc[jamming_cluster, 'cnoMean']:.2f})")
print(f"🟡 Spoofing cluster: {spoofing_cluster} (avg CNO: {cluster_stats.loc[spoofing_cluster, 'cnoMean']:.2f})")

## 5. Візуалізація результатів

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Scatter: cnoMean vs sat_efficiency
colors_map = {'Jamming': '#e74c3c', 'Spoofing': '#f1c40f'}
for attack_type, color in colors_map.items():
    subset = df_anomalies[df_anomalies['attack_type'] == attack_type]
    axes[0].scatter(
        subset['cnoMean'], subset['sat_efficiency'],
        c=color, alpha=0.3, s=10, label=attack_type
    )

axes[0].set_xlabel('CNO Mean (Signal Strength)', fontsize=12)
axes[0].set_ylabel('Satellite Efficiency', fontsize=12)
axes[0].set_title('Jamming vs Spoofing: Signal Signature', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Boxplot: cnoMean by attack type
sns.boxplot(data=df_anomalies, x='attack_type', y='cnoMean', palette=colors_map, ax=axes[1])
axes[1].set_xlabel('Attack Type', fontsize=12)
axes[1].set_ylabel('CNO Mean (dBHz)', fontsize=12)
axes[1].set_title('CNO Distribution by Attack Type', fontsize=14)

plt.tight_layout()
plt.savefig('figures/attack_clustering_results.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Статистика атак

In [ ]:
# Підрахунок
counts = df_anomalies['attack_type'].value_counts()
percentages = df_anomalies['attack_type'].value_counts(normalize=True) * 100

print("📊 Розподіл типів атак:")
print("="*40)
for attack_type in counts.index:
    print(f"   {attack_type}: {counts[attack_type]:,} ({percentages[attack_type]:.1f}%)")

# Pie chart
fig, ax = plt.subplots(figsize=(6, 6))
colors_list = [colors_map[t] for t in counts.index]
ax.pie(
    counts.values,
    labels=counts.index,
    autopct='%1.1f%%',
    colors=colors_list,
    explode=[0.05, 0],
    shadow=True,
    startangle=90
)
ax.set_title('Attack Type Distribution', fontsize=14)
plt.tight_layout()
plt.show()

## 📌 Висновки

1. **K-Means ефективно розділяє атаки** на два типи за фізичними ознаками
2. **Jamming** характеризується:
   - Низьким CNO (сигнал заглушений шумом)
   - Зниженою ефективністю супутників
3. **Spoofing** характеризується:
   - Нормальним або високим CNO (підроблений сигнал)
   - Помилками позиціонування при "хорошому" сигналі

**Практичне застосування**: Цей аналіз може бути використаний для:
- Автоматичної класифікації інцидентів
- Вибору стратегії протидії (антена vs криптографія)
- Forensic-аналізу після атаки